In [3]:
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats import weightstats as stests

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [7]:
df_before = pd.read_csv('cleaned_data/cleaned_pcd_data_before_all_parts.csv')
df_before['date_time'] = pd.to_datetime(df_before['date_time'], format = '%Y-%m-%d %H:%M')

df_before_station = pd.read_csv('./processed_data/PCD_DICTIONARY.csv')
df_before_station = df_before_station.iloc[:, 1:]
df_before_station.rename(columns = {'stationId': 'stationID', 'name': 'nameTH'}, inplace = True)

In [8]:
mapping_column = ['lat', 'long', 'province']
for c in mapping_column:
    mapping_dict = dict(df_before_station.set_index('stationID')[c])
    df_before[c] = df_before['stationID'].map(mapping_dict)
df_before['year'] = df_before['date_time'].dt.year
df_before

,stationID,date_time,CO at 3 m (ppm),NO2 at 3 m (ppb),O3 at 3 m (ppb),PM10 at 3 m (microg/m3),PM2_5 at 3 m (microg/m3),RH at 2 m (%),SO2 at 3 m (ppb),Temp at 2 m (oC),Wind_dir at 10 m (DegM),Wind_speed at 10 m (m/s),lat,long,province,year
0,35t,1995-06-04 00:00:00,NaN,NaN,NaN,NaN,NaN,86.0,NaN,27.4,249.0,NaN,18.815423,98.957566,Chiang Mai,1995
1,35t,1995-06-04 21:00:00,NaN,NaN,NaN,NaN,NaN,75.0,NaN,29.1,166.0,NaN,18.815423,98.957566,Chiang Mai,1995
2,35t,1995-06-04 22:00:00,NaN,NaN,NaN,NaN,NaN,80.0,NaN,28.4,208.0,NaN,18.815423,98.957566,Chiang Mai,1995
3,35t,1995-06-04 23:00:00,NaN,NaN,NaN,NaN,NaN,84.0,NaN,28.0,194.0,NaN,18.815423,98.957566,Chiang Mai,1995
4,35t,1995-06-05 00:00:00,NaN,NaN,NaN,NaN,NaN,87.0,NaN,27.6,168.0,NaN,18.815423,98.957566,Chiang Mai,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603462,27t,2020-09-30 23:00:00,NaN,19.0,6.0,39.0,24.0,NaN,7.0,27.9,317.0,1.1,13.549535,100.262835,Samut Sakhon,2020
5603463,26t_1,2020-09-30 23:00:00,0.42,6.0,NaN,22.0,9.0,NaN,NaN,26.3,281.0,0.6,13.523801,99.811871,Ratchaburi,2020
5603464,25t,2020-09-30 23:00:00,NaN,NaN,4.0,53.0,19.0,NaN,NaN,27.7,192.0,0.6,14.526613,100.923446,Saraburi,2020
5603465,31t,2020-09-30 23:00:00,0.48,12.0,5.0,36.0,9.0,94.0,2.0,28.3,301.0,1.0,12.732951,101.132847,Rayong,2020


In [9]:
df1 = df_before[df_before['year'] == 2017]
df1

pm25_1_avg = df1.groupby('province')['PM2_5 at 3 m (microg/m3)'].mean().reset_index(name='pm2.5_1')
pm25_1_avg.head()

,province,pm2.5_1
0,Bangkok,25.459050
1,Chachoengsao,NaN
2,Chiang Mai,26.512331
3,Chiang Rai,NaN
4,Chonburi,22.322376


In [10]:
df2 = df_before[df_before['year'] == 2018]
df2

pm25_2_avg = df2.groupby('province')['PM2_5 at 3 m (microg/m3)'].mean().reset_index(name='pm2.5_2')
pm25_2_avg.head()

,province,pm2.5_2
0,Bangkok,27.780842
1,Chachoengsao,NaN
2,Chiang Mai,29.550628
3,Chiang Rai,19.116999
4,Chonburi,24.391654


In [11]:
df3 = df_before[df_before['year'] == 2019]
df3

pm25_3_avg = df3.groupby('province')['PM2_5 at 3 m (microg/m3)'].mean().reset_index(name='pm2.5_3')
pm25_3_avg.head()

,province,pm2.5_3
0,Bangkok,25.817617
1,Chachoengsao,15.967243
2,Chiang Mai,33.415744
3,Chiang Rai,36.676824
4,Chonburi,20.838060


In [12]:
pm_avg = pm25_1_avg.merge(pm25_2_avg, on = 'province', how = 'inner')
pm_avg = pm_avg.merge(pm25_3_avg, on = 'province', how = 'inner')

pm_avg.dropna(subset=['pm2.5_1','pm2.5_2'], inplace=True)
pm_avg

,province,pm2.5_1,pm2.5_2,pm2.5_3
0,Bangkok,25.459050,27.780842,25.817617
2,Chiang Mai,26.512331,29.550628,33.415744
4,Chonburi,22.322376,24.391654,20.838060
5,Kanchanaburi,16.048112,24.440531,27.505423
6,Khon Kaen,29.390566,30.701169,34.313175
7,Lampang,19.285113,21.001932,29.736195
10,Mae Hong Son,25.123244,16.411106,26.010028
13,Nan,16.735176,16.579770,29.617945
14,Narathiwat,8.678497,15.371266,17.379350
15,Nonthaburi,24.855806,28.088469,26.137269


In [13]:
def ttest_alt(a, b, alternative='two-sided'):
    tt, tp = stats.ttest_rel(a, b)

    if alternative == 'greater':
        if tt > 0:
            tp = 1 - (1-tp)/2
        else:
            tp /= 2
    elif alternative == 'less':
        if tt <= 0:
            tp /= 2
        else:
            tp = 1 - (1-tp)/2

    return tt, tp

In [14]:
ttest,pval = ttest_alt(pm_avg['pm2.5_1'], pm_avg['pm2.5_2'], alternative='less') #between2017,2018
print(pval)

ttest,pval = ttest_alt(pm_avg['pm2.5_2'], pm_avg['pm2.5_3'], alternative='less') #between2018,2019
print(pval)

ttest,pval = ttest_alt(pm_avg['pm2.5_1'], pm_avg['pm2.5_3'], alternative='less') #between2017,2019
print(pval)

0.021602754659029284
0.03520033846066956
0.000779764794603298
